# M2177.003100 Deep Learning <br> Assignment #3 Part 3: Transformer

Copyright (C) Data Science Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. Written by Jeonghee Jo, October 2019, and modified by Jungbeom Lee, October 2020.

This is about Transformer (Vaswani et al., 2017).
https://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf

Original blog post & code:
https://github.com/Kyubyong/transformer (Tensorflow)
https://pytorch.org/tutorials/beginner/transformer_tutorial.html (PyTorch)


That said, you are allowed to copy paste the codes from the original repo.
HOWEVER, <font color=red> try to implement the model yourself first </font>, and consider the original source code as a last resort.
You will learn a lot while wrapping around your head during the implementation. And you will understand nuts and bolts of Transformers more clearly in a code level.

### Submitting your work:
<font color=red>**DO NOT clear the final outputs**</font> so that TAs can grade both your code and results.  
Once you have done **all Assignment Part 1-3**, run the *CollectSubmission.sh* script with your **Student number** as input argument. <br>
This will produce a zipped file called *[Your student number].zip*. Please submit this file on ETL. &nbsp;&nbsp; (Usage: ./*CollectSubmission.sh* 20xx-xxxxx)





### The Grading is as follows:

This assignment is an on/off one: just make this notebook **"work"** without problem by: 

1. Train your model using at least <font color=red> 12 different hyperparameter set </font>. Report performance results computed in the last code block <font color=red> for corresponding each hyperparameter set </font>. Plus, <font color=red> submit the one checkpoint file </font> of your best model. 

2. Please provide the analysis of changed hyper-parameters. (10 points)

The details are described in <font color=red>**transformer_modules.py**</font>. (There is nothing to implement in this notebook.)


Now proceed to the code.


### Hyper-parameters

In [5]:
# change hyper-parameters in this code block!


emsize = 256 # embedding dimension
nhid = 128 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.5 # the dropout value
batch_size = 32
eval_batch_size = 16
epochs = 10 # The number of epochs

### Load and Batch Data

In [6]:
# pip install torchtext==0.6.0
import torch
import torchtext
from torchtext.data.utils import get_tokenizer
TEXT = torchtext.data.Field(tokenize=get_tokenizer("basic_english"),
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)
train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)
TEXT.build_vocab(train_txt)
%env CUDA_VISIBLE_DEVICES = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ntokens = len(TEXT.vocab.stoi) # the size of vocabulary
def batchify(data, bsz):
    data = TEXT.numericalize([data.examples[0].text])
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)



env: CUDA_VISIBLE_DEVICES=0


In [7]:
from transformer_modules import *
train_data = batchify(train_txt, batch_size)
val_data = batchify(val_txt, eval_batch_size)
test_data = batchify(test_txt, eval_batch_size)

bptt = 35
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output = eval_model(data)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

In [9]:
best_val_loss = float("inf")

best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()
torch.save(best_model.state_dict(), 'models/Transformer.pth')

/home/ccw/anaconda3/envs/deep-learning-20/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:350: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


| epoch   1 |   200/ 1863 batches | lr 5.00 | ms/batch 14.33 | loss  7.92 | ppl  2760.07
| epoch   1 |   400/ 1863 batches | lr 5.00 | ms/batch 13.43 | loss  6.86 | ppl   956.05
| epoch   1 |   600/ 1863 batches | lr 5.00 | ms/batch 13.28 | loss  6.54 | ppl   695.26
| epoch   1 |   800/ 1863 batches | lr 5.00 | ms/batch 13.72 | loss  6.46 | ppl   638.69
| epoch   1 |  1000/ 1863 batches | lr 5.00 | ms/batch 13.32 | loss  6.40 | ppl   604.32
| epoch   1 |  1200/ 1863 batches | lr 5.00 | ms/batch 13.30 | loss  6.31 | ppl   551.40
| epoch   1 |  1400/ 1863 batches | lr 5.00 | ms/batch 13.32 | loss  6.26 | ppl   521.18
| epoch   1 |  1600/ 1863 batches | lr 5.00 | ms/batch 13.32 | loss  6.22 | ppl   500.32
| epoch   1 |  1800/ 1863 batches | lr 5.00 | ms/batch 13.30 | loss  6.21 | ppl   496.45
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 26.27s | valid loss  5.98 | valid ppl   396.56
-----------------------------------

| epoch   9 |   200/ 1863 batches | lr 3.15 | ms/batch 14.55 | loss  5.54 | ppl   254.63
| epoch   9 |   400/ 1863 batches | lr 3.15 | ms/batch 13.83 | loss  5.52 | ppl   248.57
| epoch   9 |   600/ 1863 batches | lr 3.15 | ms/batch 13.81 | loss  5.49 | ppl   243.46
| epoch   9 |   800/ 1863 batches | lr 3.15 | ms/batch 13.81 | loss  5.52 | ppl   249.54
| epoch   9 |  1000/ 1863 batches | lr 3.15 | ms/batch 13.77 | loss  5.53 | ppl   252.47
| epoch   9 |  1200/ 1863 batches | lr 3.15 | ms/batch 13.81 | loss  5.53 | ppl   251.57
| epoch   9 |  1400/ 1863 batches | lr 3.15 | ms/batch 13.80 | loss  5.46 | ppl   235.45
| epoch   9 |  1600/ 1863 batches | lr 3.15 | ms/batch 13.76 | loss  5.48 | ppl   238.76
| epoch   9 |  1800/ 1863 batches | lr 3.15 | ms/batch 13.81 | loss  5.50 | ppl   243.78
-----------------------------------------------------------------------------------------
| end of epoch   9 | time: 27.07s | valid loss  5.68 | valid ppl   293.20
-----------------------------------

In [10]:
best_model_loaded = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)
best_model_loaded.load_state_dict(torch.load('models/Transformer.pth'), strict=True)
test_loss = evaluate(best_model_loaded, test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

| End of training | test loss  5.58 | test ppl   266.23


In [3]:
%%HTML
<style>
td{font-size: 12px}
</style>

## Analysis on each hyper-parameter.

The experiment results for each parameter set are summarized as table below.

| emsize | nhid | nlayers | nhead | dropout | test_loss |
|--------|------|---------|-------|---------|-----------|
| 256    | 128  | 2       | 2     | 0.5     | 5.58      |
|        |      |         |       |         |           |
|        |      |         |       |         |           |
|        |      |         |       |         |           |
|        |      |         |       |         |           |
|        |      |         |       |         |           |
|        |      |         |       |         |           |
|        |      |         |       |         |           |
|        |      |         |       |         |           |
|        |      |         |       |         |           |
|        |      |         |       |         |           |
|        |      |         |       |         |           |

